In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy import stats
from helper import *
from scipy.stats.mstats import gmean
#Importation of all the packages
from datetime import datetime
import math
import json
import zipfile  
import ssl
from scipy import stats
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn import metrics

#To dowload data
import requests
import io
import gzip

#To create the mapchart
import iso3166
import plotly
from iso3166 import countries
import plotly.graph_objects as go
import plotly.express as px

In [23]:
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")
pageview_df_imp_country = pageview_df[["date"] + list(get_country_dict().values())].set_index('date')
pageview_df_imp_country.head()
country_dict = get_country_dict()
inv_country_dict = {v: k for k, v in country_dict.items()}

In [7]:
ssl._create_default_https_context = ssl._create_unverified_context
death_url = "https://github.com/owid/covid-19-data/blob/master/public/data/jhu/new_deaths.csv?raw=true" # Make sure the url is the raw version of the file on GitHub
cases_url = "https://github.com/owid/covid-19-data/blob/master/public/data/jhu/new_cases.csv?raw=true"

# Reading the downloaded content and turning it into a pandas dataframe
deaths = pd.read_csv(death_url,index_col=0)
cases = pd.read_csv(cases_url,index_col=0)
deaths= deaths.interpolate(method ='linear', limit_direction ='forward') 
cases = cases.interpolate(method ='linear', limit_direction ='forward') 
deaths = deaths.fillna(0)
cases = cases.fillna(0)
#Keep only values until July
deaths = deaths[deaths.index < '2022-01-01']
cases = cases[cases.index < '2022-01-01']

In [35]:
covid_death = pd.DataFrame({})
for country in list(country_dict.keys()):
    if (country in countries):
        df = pd.DataFrame(deaths[country])
        df = df.rename(columns = {country : 'deaths'})
        df['Country_code'] = [countries.get(country).alpha3] * len(df)
        df['date'] = df.index
        #To keep every 10 rows
        df = df.iloc[::10,:]
        covid_death = covid_death.append(df)


fig = px.choropleth(covid_death, locations= "Country_code",
                    color = covid_death['deaths'],
                    animation_frame='date',
                    color_continuous_midpoint = max(covid_death['deaths']/2),
color_continuous_scale=px.colors.sequential.thermal_r)
#fig.update_layout(margin=dict(l=20,r=0,b=0,t=70,pad=0),paper_bgcolor="white",height= 700,title_text = 'Number of daily COVID-19 cases worldwide',font_size=18)
#fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 10
#fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.update_layout(title={'text':'Number of death of COVID-19 by Date',
                         'font':{'size':18, 'color':'red'},
                         'x':0.5, 'y':1},
                  transition = {'duration': 10})
fig.show()

<ipython-input-35-0dbcafb8cd2d>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-35-0dbcafb8cd2d>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-35-0dbcafb8cd2d>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-35-0dbcafb8cd2d>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-35-0dbcafb8cd2d>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-35-0dbcafb8cd2d>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

            deaths Country_code        date
date                                       
2020-01-22     0.0          ITA  2020-01-22
2020-02-01     0.0          ITA  2020-02-01
2020-02-11     0.0          ITA  2020-02-11
2020-02-21     1.0          ITA  2020-02-21
2020-03-02    18.0          ITA  2020-03-02
...            ...          ...         ...
2021-11-12     9.0          MNG  2021-11-12
2021-11-22     8.0          MNG  2021-11-22
2021-12-02     3.0          MNG  2021-12-02
2021-12-12     5.0          MNG  2021-12-12
2021-12-22     3.0          MNG  2021-12-22

[2414 rows x 3 columns]


In [24]:
df_pageviews_pearson = pageview_df_imp_country.loc[pageview_df_imp_country.index < '2022-01-01']
df_pageviews_pearson = df_pageviews_pearson.interpolate(method ='linear', limit_direction ='forward') 
df_pageviews_pearson = df_pageviews_pearson.fillna(0)

In [37]:
pageviews = pd.DataFrame({})
for country in country_dict.items():
    if (country[0] in countries):
        df = pd.DataFrame(df_pageviews_pearson[country[1]])
        df = df.rename(columns = {country[1] : 'pages'})
        df['Country_code'] = [countries.get(country[0]).alpha3] * len(df)
        df['date'] = df.index
        #To keep every 10 rows
        df = df.iloc[::10,:]
        pageviews = pageviews.append(df)
print(pageviews)


fig = px.choropleth(pageviews, locations= "Country_code",
                    color = pageviews['pages'],
                    animation_frame='date',
                    color_continuous_midpoint = max(pageviews['pages']/2),
color_continuous_scale=px.colors.sequential.thermal_r)
#fig.update_layout(margin=dict(l=20,r=0,b=0,t=70,pad=0),paper_bgcolor="white",height= 700,title_text = 'Number of daily COVID-19 cases worldwide',font_size=18)
#fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 10
#fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.update_layout(title={'text':'Number of pageviews',
                         'font':{'size':18, 'color':'red'},
                         'x':0.5, 'y':1},
                  transition = {'duration': 10})
fig.show()

<ipython-input-37-98a122e2f4c1>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-37-98a122e2f4c1>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-37-98a122e2f4c1>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-37-98a122e2f4c1>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-37-98a122e2f4c1>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-37-98a122e2f4c1>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

              pages Country_code        date
date                                        
2020-01-01    252.0          ITA  2020-01-01
2020-01-11   3094.0          ITA  2020-01-11
2020-01-21  29010.0          ITA  2020-01-21
2020-01-31  90986.0          ITA  2020-01-31
2020-02-10   9809.0          ITA  2020-02-10
...             ...          ...         ...
2021-11-21     16.0          MNG  2021-11-21
2021-12-01     17.0          MNG  2021-12-01
2021-12-11      9.0          MNG  2021-12-11
2021-12-21      3.0          MNG  2021-12-21
2021-12-31      1.0          MNG  2021-12-31

[2516 rows x 3 columns]
